In [1]:
# import itertools
import numpy as np
from MLP import *
import os
import time
import itertools
from sklearn.metrics import accuracy_score, recall_score, f1_score
import csv
import concurrent.futures
import json
import pickle


c:\Users\Sebyb\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\Sebyb\anaconda3\lib\site-packages\numpy\.libs\libopenblas.4SP5SUA7CBGXUEOC35YP2ASOICYYEQZZ.gfortran-win_amd64.dll
c:\Users\Sebyb\anaconda3\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [29]:
import numpy as np
from MLP import *
import os
import time
import itertools
from sklearn.metrics import accuracy_score, recall_score, f1_score
import csv
import concurrent.futures
import json
import pickle
import traceback

X = np.load('Assignment1-Dataset/train_data.npy')
labels = np.load('Assignment1-Dataset/train_label.npy')
y = np.array([MLP.class_to_one_hot(label, 10) for label in labels])

# Seed for reproducibility
np.random.seed(0)

# 5-fold Cross Validation
n_samples = X.shape[0]
n_folds = 5
indices = np.arange(n_samples)
np.random.shuffle(indices)

fold_sizes = np.full(n_folds, n_samples // n_folds, dtype=int)
fold_sizes[:n_samples % n_folds] += 1
current = 0
folds = []
for fold_size in fold_sizes:
    start, stop = current, current + fold_size
    folds.append(indices[start:stop])
    current = stop

# Generate training and validation datasets for each fold
fold_splits = []
for i in range(n_folds):
        # Generate training and validation sets for this fold
        val_indices = folds[i]
        train_indices = np.hstack(folds[:i] + folds[i+1:])
        
        X_train, y_train = X[train_indices], y[train_indices]
        X_val, y_val = X[val_indices], y[val_indices]
        fold_splits.append((X_train, y_train, X_val, y_val))
        
# Make copies of the fold_splits data for async processing
fold_splits = [(X_train.copy(), y_train.copy(), X_val.copy(), y_val.copy()) for (X_train, y_train, X_val, y_val) in fold_splits]

fold_splits = fold_splits[:1]

def can_be_pickled(obj):
    try:
        pickle.dumps(obj)
        return True
    except (pickle.PicklingError, TypeError) as e:
        print(f"Cannot be pickled: {e}")
        return False

print(can_be_pickled(fold_splits))
    
### Constants ###

SETUP = {
    'epochs': 2,
    'activations': [None, 'ReLU', 'ReLU', 'softmax'],
    'input_size': 128,
    'early_stopping': (10, 0.001)
}

### Options for Hyper-Parameters ###

weight_decay_options = [0.0, 0.0001, 0.001]
drop_out_options = [[0.0, 0.0, 0.0, 0.0], [0.5, 0.2, 0.2, 0.0], [0.1, 0.3, 0.3, 0.1]]
hidden_layer_options = [[128, 64, 32, 10]]
lr_options = [0.001, 0.0001, 0.01]
optimiser_options = [None, 'Adam', 'Momentum']
bn_option = [False, True]
batch_size_options = [10,5,2]


def log_detailed_metrics(hyperparams_dict, cv_metrics, filename="detailed_model_performance.csv"):
    # Check if file exists, if not, write headers
    file_exists = os.path.isfile(filename)
    
    with open(filename, mode='a', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        if not file_exists:
            writer.writerow(["Fold", "Hyperparameters", "Train Losses", "Val Losses", "Val Accuracy", "Val Recall", "Val F1", "Time Taken", "Early Stop Epoch"])

        for i, metrics in enumerate(cv_metrics):
            # Serialize detailed metrics as JSON strings
            detailed_train_loss = json.dumps(metrics['detailed_train_loss'])
            detailed_val_loss = json.dumps(metrics['detailed_val_loss'])
            writer.writerow([
                i + 1,
                json.dumps(hyperparams_dict),  # Hyperparameters as JSON
                detailed_train_loss,
                detailed_val_loss,
                metrics['val_accuracy'],
                metrics['val_recall'],
                metrics['val_f1'],
                metrics['time_taken'],
                metrics['early_stop_epoch']
            ])

def train_and_evaluate_fold(X_train, y_train, X_val, y_val, hyperparams_dict, static_params):
    """
    Trains the model for one fold and evaluates it.
    This function is designed to be run in a separate process.
    """
    
    return {
        'train_loss': 1,
        'val_loss': 1,
        'val_accuracy': 1,
        'val_recall': 1,
        'val_f1': 1,
        'time_taken': 1,
        'early_stop_epoch': 1,
        'detailed_train_loss': 1,
        'detailed_val_loss': 1
    }
    # log_path = "process_log.txt"
    # with open(log_path, "a") as log_file:
    #     log_file.write(f"Starting fold with params: {hyperparams_dict}\n")
    
    try:
        
        hidden_layers = hyperparams_dict['hidden_layers']
        lr = hyperparams_dict['lr']
        batch_size = hyperparams_dict['batch_size']
        optimiser = hyperparams_dict['optimiser']
        bn = hyperparams_dict['bn']
        weight_decay = hyperparams_dict['weight_decay']
        dropout = hyperparams_dict['dropout']
        
        print('Starting a fold with hyperparameters:', hyperparams_dict)
        # Initialize MLP model with current hyperparameters
        nn = MLP(hidden_layers, static_params['activations'], bn, weight_decay, dropout)

        # Fit the model
        start = time.time()
        nn_output = nn.fit(X_train, y_train, X_val, y_val, learning_rate=lr, epochs=static_params['epochs'], batch_size=batch_size, optimiser=optimiser, early_stopping=static_params['early_stopping'])
        end = time.time()

        # Extract metrics
        train_loss, val_loss, early_stop_epoch = nn_output
        time_taken = end - start

        # Predict on validation set
        output_val = nn.predict(X_val)

        # Calculate metrics
        val_accuracy = accuracy_score(np.argmax(y_val, axis=1), np.argmax(output_val, axis=1))
        val_recall = recall_score(np.argmax(y_val, axis=1), np.argmax(output_val, axis=1), average='macro')
        val_f1 = f1_score(np.argmax(y_val, axis=1), np.argmax(output_val, axis=1), average='macro')
        
        # Print metrics
        print(f"Train Loss: {train_loss[-1]}, Val Loss: {val_loss[-1]}, Val Accuracy: {val_accuracy}, Val Recall: {val_recall}, Val F1: {val_f1}, Time Taken: {time_taken}")

        return {
            'train_loss': train_loss[-1],
            'val_loss': val_loss[-1],
            'val_accuracy': val_accuracy,
            'val_recall': val_recall,
            'val_f1': val_f1,
            'time_taken': time_taken,
            'early_stop_epoch': early_stop_epoch,
            'detailed_train_loss': train_loss,
            'detailed_val_loss': val_loss
        }
    except Exception as e:
        with open('process_log.txt', "a") as log_file:
            log_file.write(f"Error: {e}\n")
        return {
            'train_loss': None,
            'val_loss': None,
            'val_accuracy': None,
            'val_recall': None,
            'val_f1': None,
            'time_taken': None,
            'early_stop_epoch': None,
            'detailed_train_loss': None,
            'detailed_val_loss': None
        }
        

# Function to run your model CV asynchronously
def run_model_cv_async(fold_splits, hyperparams_dict, static_params):
    results = []
    # with concurrent.futures.ProcessPoolExecutor(max_workers=10) as executor:
    #     # Schedule the execution of each fold
    #     print('Sending worker')
    #     futures = [executor.submit(train_and_evaluate_fold, X_train, y_train, X_val, y_val, hyperparams_dict)
    #                for (X_train, y_train, X_val, y_val) in fold_splits]

    #     for future in concurrent.futures.as_completed(futures):
    #         results.append(future.result())
            
    with concurrent.futures.ProcessPoolExecutor(max_workers=5) as executor:
        futures = []
        for X_train, y_train, X_val, y_val in fold_splits:
            print('Sending worker')
            print(f'X_train: {can_be_pickled(X_train)}, y_train: {can_be_pickled(y_train)}, X_val: {can_be_pickled(X_val)}, y_val: {can_be_pickled(y_val)}, hyperparams_dict: {can_be_pickled(hyperparams_dict)}, static_params: {can_be_pickled(static_params)}')
            # Correctly pass the slices of your dataset and hyperparameters to the function
            future = executor.submit(train_and_evaluate_fold, X_train, y_train, X_val, y_val, hyperparams_dict, static_params)
            futures.append(future)
            print('Worker sent')

        for future in concurrent.futures.as_completed(futures):
            results.append(future.result())
            print('Worker finished')

    return results
    
    print('Running in serial mode')
    for (X_train, y_train, X_val, y_val) in fold_splits:
        results.append(train_and_evaluate_fold(X_train, y_train, X_val, y_val, hyperparams_dict))

    # After all futures are completed, results are collected
    # Now, you can aggregate or process results as needed
    return results

True


In [30]:
outputs = []

print('Begin Hyper-parameter Tuning -')
for hyperparams in itertools.product(weight_decay_options, drop_out_options, hidden_layer_options, lr_options, optimiser_options, bn_option, batch_size_options):
    hyperparams_dict = {
        'weight_decay': hyperparams[0],
        'dropout': hyperparams[1],
        'hidden_layers': hyperparams[2],
        'lr': hyperparams[3],
        'optimiser': hyperparams[4],
        'bn': hyperparams[5],
        'batch_size': hyperparams[6]
    }
    print(f'\n\n### Running model {str(hyperparams_dict)}')
    print(can_be_pickled(hyperparams_dict))
    try:
        start = time.time()
        # Use the asynchronous version here
        cv_metrics = run_model_cv_async(fold_splits, hyperparams_dict, SETUP)
        end = time.time()
        time_taken = end - start
        
        # Process and log the cv_metrics as needed
        # Ensure cv_metrics are aggregated properly from the async results
        
        log_detailed_metrics(hyperparams_dict, cv_metrics, time_taken)

    except Exception as e:
        print(f'Error: {e}')
        traceback_str = traceback.format_exc()  # This gives you the full traceback as a string
        print(f'Error: {traceback_str}')
        # try:
        #     log_detailed_metrics(hyperparams_dict, None, 99999)  # Log a dummy value in case of error
        # except:
        #     pass



Begin Hyper-parameter Tuning -


### Running model {'weight_decay': 0.0, 'dropout': [0.0, 0.0, 0.0, 0.0], 'hidden_layers': [128, 64, 32, 10], 'lr': 0.001, 'optimiser': None, 'bn': False, 'batch_size': 10}
True
Sending worker
X_train: True, y_train: True, X_val: True, y_val: True, hyperparams_dict: True, static_params: True
Worker sent
Error: A process in the process pool was terminated abruptly while the future was running or pending.
Error: Traceback (most recent call last):
  File "C:\Users\Sebyb\AppData\Local\Temp\ipykernel_33764\1252734569.py", line 19, in <module>
    cv_metrics = run_model_cv_async(fold_splits, hyperparams_dict, SETUP)
  File "C:\Users\Sebyb\AppData\Local\Temp\ipykernel_33764\4239297946.py", line 211, in run_model_cv_async
    results.append(future.result())
  File "c:\Users\Sebyb\anaconda3\lib\concurrent\futures\_base.py", line 439, in result
    return self.__get_result()
  File "c:\Users\Sebyb\anaconda3\lib\concurrent\futures\_base.py", line 391, in __get_resu

KeyboardInterrupt: 

In [31]:
import concurrent.futures

def simple_function(x):
    return x * x

with concurrent.futures.ProcessPoolExecutor(max_workers=5) as executor:
    futures = [executor.submit(simple_function, i) for i in range(10)]
    for future in concurrent.futures.as_completed(futures):
        print(future.result())

BrokenProcessPool: A process in the process pool was terminated abruptly while the future was running or pending.

In [3]:
### Constants ###

# SETUP = {
#     'epochs': 40,
#     'activations': [None, 'ReLU', 'ReLU', 'softmax'],
#     'input_size': 128,
#     'early_stopping': (10, 0.001)
# }

# ### Options for Hyper-Parameters ###

# hidden_layer_options = [[128, 64, 32, 10], [128, 96, 64, 10]]
# lr_options = [0.001, 0.0001, 0.01]
# batch_size_options = [128, 64, 16, 4]
# optimiser_options = [None, 'Adam', 'Momentum']
# bn_option = [False, True]
# weight_decay_options = [0.0, 0.0001, 0.001]
# drop_out_options = [[0.0, 0.0, 0.0, 0.0], [0.0, 0.2, 0.2, 0.0], [0.2, 0.3, 0.3, 0.2], [0.0, 0.4, 0.4, 0.0]]

# lr_options = [0.005, 0.001, 0.0001]
# batch_size_options = [4,8,16]
# optimiser_options = ['Adam', 'Momentum'] # None as well
# bn_option = [False, True]

# dropout_options = []
# size_options = []
# weight_decay_options
# activation_options = ['ReLU', 'tanh']

In [4]:
def log_to_file(hyperparams_dict, cv_metrics, time_taken, filename="model_performance_2.csv"):
    # Check if file exists, if not, write headers
    file_exists = os.path.isfile(filename)
    hidden_layers = hyperparams_dict['hidden_layers']
    lr = hyperparams_dict['lr']
    batch_size = hyperparams_dict['batch_size']
    optimiser = hyperparams_dict['optimiser']
    bn = hyperparams_dict['bn']
    weight_decay = hyperparams_dict['weight_decay']
    dropout = hyperparams_dict['dropout']
    
    with open(filename, mode='a', newline='') as file:
        writer = csv.writer(file)
        if not file_exists:
            # Write the header
            # CV Metrics structure cv_train_loss_scores, cv_val_loss_scores, cv_val_accuracy_scores, cv_val_recall_scores, cv_val_f1_scores, cv_times, detailed_train_losses, detailed_val_losses
            
            writer.writerow(["Train Loss", "Val Loss", "Val Accuracy", "Val Recall", "Val F1", "Learning Rate", "Batch Size", "Optimizer", "Batch Norm", "Weight Decay", "Dropout", "Hidden Layers", "Time Taken Test", "Time Taken CV", "Train Losses CV", "Val Losses CV"])
        
        # Write the data
        for i, (cv_train_loss_score, cv_val_loss_score, cv_val_accuracy_score, cv_val_recall_score, cv_val_f1_score, cv_time, detailed_train_loss, detailed_val_loss) in enumerate(zip(*cv_metrics)):
            # writer.writerow([hyperparams[0], hyperparams[1], hyperparams[2], hyperparams[3], hyperparams[4], hyperparams[5], hyperparams[6], cv_train_loss_scores, cv_val_loss_scores, cv_val_accuracy_scores, cv_val_recall_scores, cv_val_f1_scores, time_taken, cv_times, detailed_train_losses, detailed_val_losses])
            # writer.writerow([round(cv_train_loss_scores, 4), round(cv_val_loss_scores, 4), round(cv_val_accuracy_scores, 4), round(cv_val_recall_scores, 4), round(cv_val_f1_scores, 4), lr, batch_size, optimiser, bn, weight_decay, dropout, hidden_layers, time_taken, cv_times, detailed_train_losses, detailed_val_losses])
            writer.writerow([\
                round(cv_train_loss_score, 4), 
                round(cv_val_loss_score, 4), 
                round(cv_val_accuracy_score, 4), 
                round(cv_val_recall_score, 4), 
                round(cv_val_f1_score, 4), 
                hyperparams_dict['lr'], 
                hyperparams_dict['batch_size'], 
                hyperparams_dict['optimiser'], 
                hyperparams_dict['bn'], 
                hyperparams_dict['weight_decay'], 
                hyperparams_dict['dropout'], 
                hyperparams_dict['hidden_layers'], 
                time_taken, 
                cv_time,
                detailed_train_loss, 
                detailed_val_loss
            ])

In [7]:
### Constants ###

SETUP = {
    'epochs': 50,
    'activations': [None, 'ReLU', 'ReLU', 'softmax'],
    'input_size': 128,
    'early_stopping': (10, 0.001)
}

### Options for Hyper-Parameters ###

weight_decay_options = [0.0, 0.0001, 0.001]
drop_out_options = [[0.0, 0.0, 0.0, 0.0], [0.5, 0.2, 0.2, 0.0], [0.1, 0.3, 0.3, 0.1]]
hidden_layer_options = [[128, 64, 32, 10], [128, 96, 64, 10]]
lr_options = [0.001, 0.0001, 0.01]
optimiser_options = [None, 'Adam', 'Momentum']
bn_option = [False, True]
batch_size_options = [16,8,4,2]

In [8]:
def log_detailed_metrics(hyperparams_dict, cv_metrics, filename="detailed_model_performance.csv"):
    # Check if file exists, if not, write headers
    file_exists = os.path.isfile(filename)
    
    with open(filename, mode='a', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        if not file_exists:
            writer.writerow(["Fold", "Hyperparameters", "Train Losses", "Val Losses", "Val Accuracy", "Val Recall", "Val F1", "Time Taken", "Early Stop Epoch"])

        for i, metrics in enumerate(cv_metrics):
            # Serialize detailed metrics as JSON strings
            detailed_train_loss = json.dumps(metrics['detailed_train_loss'])
            detailed_val_loss = json.dumps(metrics['detailed_val_loss'])
            writer.writerow([
                i + 1,
                json.dumps(hyperparams_dict),  # Hyperparameters as JSON
                detailed_train_loss,
                detailed_val_loss,
                metrics['val_accuracy'],
                metrics['val_recall'],
                metrics['val_f1'],
                metrics['time_taken'],
                metrics['early_stop_epoch']
            ])

In [9]:
def train_and_evaluate_fold(X_train, y_train, X_val, y_val, hyperparams_dict):
    """
    Trains the model for one fold and evaluates it.
    This function is designed to be run in a separate process.
    """
    hidden_layers, lr, batch_size = hyperparams_dict['hidden_layers'], hyperparams_dict['lr'], hyperparams_dict['batch_size']
    optimiser, bn, weight_decay, dropout = hyperparams_dict['optimiser'], hyperparams_dict['bn'], hyperparams_dict['weight_decay'], hyperparams_dict['dropout']

    # Initialize MLP model with current hyperparameters
    nn = MLP(hidden_layers, SETUP['activations'], bn, weight_decay, dropout)

    # Fit the model
    start = time.time()
    nn_output = nn.fit(X_train, y_train, X_val, y_val, learning_rate=lr, epochs=SETUP['epochs'], batch_size=batch_size, optimiser=optimiser, early_stopping=SETUP['early_stopping'])
    end = time.time()

    # Extract metrics
    train_loss, val_loss, early_stop_epoch = nn_output
    time_taken = end - start

    # Predict on validation set
    output_val = nn.predict(X_val)

    # Calculate metrics
    val_accuracy = accuracy_score(np.argmax(y_val, axis=1), np.argmax(output_val, axis=1))
    val_recall = recall_score(np.argmax(y_val, axis=1), np.argmax(output_val, axis=1), average='macro')
    val_f1 = f1_score(np.argmax(y_val, axis=1), np.argmax(output_val, axis=1), average='macro')
    
    # Print metrics
    print(f"Train Loss: {train_loss[-1]}, Val Loss: {val_loss[-1]}, Val Accuracy: {val_accuracy}, Val Recall: {val_recall}, Val F1: {val_f1}, Time Taken: {time_taken}")

    return {
        'train_loss': train_loss[-1],
        'val_loss': val_loss[-1],
        'val_accuracy': val_accuracy,
        'val_recall': val_recall,
        'val_f1': val_f1,
        'time_taken': time_taken,
        'early_stop_epoch': early_stop_epoch,
        'detailed_train_loss': train_loss,
        'detailed_val_loss': val_loss
    }

# Function to run your model CV asynchronously
def run_model_cv_async(fold_splits, hyperparams_dict):
    results = []
    with concurrent.futures.ProcessPoolExecutor() as executor:
        # Schedule the execution of each fold
        futures = [executor.submit(train_and_evaluate_fold, X_train, y_train, X_val, y_val, hyperparams_dict)
                   for (X_train, y_train, X_val, y_val) in fold_splits]

        for future in concurrent.futures.as_completed(futures):
            results.append(future.result())

    # After all futures are completed, results are collected
    # Now, you can aggregate or process results as needed
    return results

# # Example usage:
# # Ensure you have defined `fold_splits` and `hyperparams_dict` appropriately
# results = run_model_cv_async(fold_splits, hyperparams_dict)
# for result in results:
#     print(result)

In [10]:
# Seed for reproducibility
np.random.seed(0)

# Generate training and validation datasets for each fold
fold_splits = []
for i in range(n_folds):
        # Generate training and validation sets for this fold
        val_indices = folds[i]
        train_indices = np.hstack(folds[:i] + folds[i+1:])
        
        X_train, y_train = X[train_indices], y[train_indices]
        X_val, y_val = X[val_indices], y[val_indices]
        fold_splits.append((X_train, y_train, X_val, y_val))

outputs = []

print('Begin Hyper-parameter Tuning -')
for hyperparams in itertools.product(weight_decay_options, drop_out_options, hidden_layer_options, lr_options, optimiser_options, bn_option, batch_size_options):
    hyperparams_dict = {
        'weight_decay': hyperparams[0],
        'dropout': hyperparams[1],
        'hidden_layers': hyperparams[2],
        'lr': hyperparams[3],
        'optimiser': hyperparams[4],
        'bn': hyperparams[5],
        'batch_size': hyperparams[6]
    }
    print(f'\n\n### Running model {str(hyperparams_dict)}')
    try:
        start = time.time()
        # Use the asynchronous version here
        cv_metrics = run_model_cv_async(fold_splits, hyperparams_dict)
        end = time.time()
        time_taken = end - start
        
        # Process and log the cv_metrics as needed
        # Ensure cv_metrics are aggregated properly from the async results
        
        log_detailed_metrics(hyperparams_dict, cv_metrics, time_taken)

    except Exception as e:
        print(f'Error: {e}')
        try:
            log_detailed_metrics(hyperparams_dict, None, 99999)  # Log a dummy value in case of error
        except:
            pass


Begin Hyper-parameter Tuning -


### Running model {'weight_decay': 0.0, 'dropout': [0.0, 0.0, 0.0, 0.0], 'hidden_layers': [128, 64, 32, 10], 'lr': 0.001, 'optimiser': None, 'bn': False, 'batch_size': 16}


In [6]:
# Function to run your model
def run_model_cv(fold_splits, hyperparams_dict):
    # weight_decay_options, drop_out_options, hidden_layer_options, lr_options, optimiser_options, bn_option, batch_size_options
    cv_train_loss_scores = []
    cv_val_loss_scores = []
    detailed_train_losses = []
    detailed_val_losses = []
    cv_val_accuracy_scores = []
    cv_val_recall_scores = []
    cv_val_f1_scores = []
    cv_times = []
    hidden_layers, lr, batch_size = hyperparams_dict['hidden_layers'], hyperparams_dict['lr'], hyperparams_dict['batch_size']
    optimiser, bn, weight_decay, dropout = hyperparams_dict['optimiser'], hyperparams_dict['bn'], hyperparams_dict['weight_decay'], hyperparams_dict['dropout']
    
    for i in range(n_folds):
        X_train, y_train, X_val, y_val = fold_splits[i]
        
        # Initialize MLP model with current hyperparameters
        nn = MLP(hidden_layers, SETUP['activations'], bn, weight_decay, dropout)
        
        # Fit the model
        start = time.time()
        nn_output = nn.fit(X_train, y_train, X_val, y_val, learning_rate=lr, epochs=SETUP['epochs'], batch_size=batch_size, optimiser=optimiser, early_stopping=SETUP['early_stopping'])
        end = time.time()
        
        # Extract metrics
        train_loss, val_loss, early_stop_epoch = nn_output
        time_taken = end - start
        cv_times.append(time_taken)
        
        # Predict on validation set
        output_val = nn.predict(X_val)
        
        # Calulate metrics
        val_accuracy = accuracy_score(np.argmax(y_val, axis=1), np.argmax(output_val, axis=1))
        val_recall = recall_score(np.argmax(y_val, axis=1), np.argmax(output_val, axis=1), average='macro')
        val_f1 = f1_score(np.argmax(y_val, axis=1), np.argmax(output_val, axis=1), average='macro')
        
        # Append metrics for this fold
        cv_train_loss_scores.append(train_loss[-1])
        cv_val_loss_scores.append(val_loss[-1])
        detailed_train_losses.append(train_loss)
        detailed_val_losses.append(val_loss)
        cv_val_accuracy_scores.append(val_accuracy)
        cv_val_recall_scores.append(val_recall)
        cv_val_f1_scores.append(val_f1)

        print(f'CV_{i+1} ({time_taken:.2f}s) -\t train_loss: {train_loss[-1]:.5f}\t val_loss: {val_loss[-1]:.5f}\t val_acc: {val_accuracy:.5f}\t val_recall: {val_recall:.5f}\t val_f1: {val_f1:.5f}')
    
    return cv_train_loss_scores, cv_val_loss_scores, cv_val_accuracy_scores, cv_val_recall_scores, cv_val_f1_scores, cv_times, detailed_train_losses, detailed_val_losses

In [8]:
# Seed for reproducibility
np.random.seed(0)

# Generate training and validation datasets for each fold
fold_splits = []
for i in range(n_folds):
        # Generate training and validation sets for this fold
        val_indices = folds[i]
        train_indices = np.hstack(folds[:i] + folds[i+1:])
        
        X_train, y_train = X[train_indices], y[train_indices]
        X_val, y_val = X[val_indices], y[val_indices]
        fold_splits.append((X_train, y_train, X_val, y_val))

outputs = []

print('Begin Hyper-parameter Tuning -')
for hyperparams in itertools.product(weight_decay_options, drop_out_options, hidden_layer_options, lr_options, optimiser_options, bn_option, batch_size_options):
        # Convert hyperparams to dictionary for easier access
        hyperparams_dict = {
                'weight_decay': hyperparams[0],
                'dropout': hyperparams[1],
                'hidden_layers': hyperparams[2],
                'lr': hyperparams[3],
                'optimiser': hyperparams[4],
                'bn': hyperparams[5],
                'batch_size': hyperparams[6]
        }
        print(f'\n\n### Running model {str(hyperparams_dict)}')
        try:
                start = time.time()
                cv_metrics = run_model_cv(fold_splits, hyperparams_dict)
                end = time.time()
                time_taken = end-start
                
                cv_train_loss_scores, cv_val_loss_scores, cv_val_accuracy_scores, cv_val_recall_scores, \
                        cv_val_f1_scores, cv_times, detailed_train_losses, detailed_val_losses = cv_metrics
                train_loss = np.mean(cv_train_loss_scores)
                val_loss = np.mean(cv_val_loss_scores)
                val_accuracy = np.mean(cv_val_accuracy_scores)
                val_recall = np.mean(cv_val_recall_scores)
                val_f1 = np.mean(cv_val_f1_scores)
                
                print(f'## Experiment results ({time_taken:.2f}s) - {str(hyperparams_dict)}')
                print(f'# Metrics - train loss: {train_loss:.5f} \t val loss: {val_loss:.5f} \t val accuracy: {val_accuracy:.5f} \t val recall: {val_recall:.5f} \t val f1: {val_f1:.5f}')
                # Print the experiment results string and print to file
                
                log_to_file(hyperparams_dict, cv_metrics, time_taken)
                
        except Exception as e:
                print(f'Error: {e}')
                log_to_file(hyperparams_dict, [[0]*n_folds]*8, 99999)
                continue

Begin Hyper-parameter Tuning -


### Running model {'weight_decay': 0.0, 'dropout': [0.0, 0.0, 0.0, 0.0], 'hidden_layers': [128, 64, 32, 10], 'lr': 0.001, 'optimiser': None, 'bn': False, 'batch_size': 16}
	early stopping at ep.17	 train loss: 0.88703	 val loss: 2.44500	 best val loss: 2.37403
CV_1 (7.34s) -	 train_loss: 0.88703	 val_loss: 2.44500	 val_acc: 0.33450	 val_recall: 0.33385	 val_f1: 0.32795
	early stopping at ep.11	 train loss: 0.78747	 val loss: 2.82258	 best val loss: 2.37687
CV_2 (5.21s) -	 train_loss: 0.78747	 val_loss: 2.82258	 val_acc: 0.32850	 val_recall: 0.32779	 val_f1: 0.32160
	early stopping at ep.30	 train loss: 0.83872	 val loss: 2.44399	 best val loss: 2.38876
CV_3 (12.88s) -	 train_loss: 0.83872	 val_loss: 2.44399	 val_acc: 0.35430	 val_recall: 0.35525	 val_f1: 0.34826
	early stopping at ep.12	 train loss: 0.82074	 val loss: 2.47603	 best val loss: 2.46724
CV_4 (5.09s) -	 train_loss: 0.82074	 val_loss: 2.47603	 val_acc: 0.34810	 val_recall: 0.34858	 val_f1: 0.

In [64]:
# Seed for reproducibility
np.random.seed(0)

# Generate training and validation datasets for each fold
fold_splits = []
for i in range(n_folds):
        # Generate training and validation sets for this fold
        val_indices = folds[i]
        train_indices = np.hstack(folds[:i] + folds[i+1:])
        
        X_train, y_train = X[train_indices], y[train_indices]
        X_val, y_val = X[val_indices], y[val_indices]
        fold_splits.append((X_train, y_train, X_val, y_val))

outputs = []

print('Begin Hyper-parameter Tuning -')
for hyperparams in itertools.product(lr_options, batch_size_options, optimiser_options, bn_option):
        print(f'\n\n### Running model {str(hyperparams)}')
        start = time.time()
        cv_metrics = run_model_cv(fold_splits, hyperparams)
        end = time.time()
        time_took = end-start
        
        train_losses, val_losses, val_accuracies, times = cv_metrics
        train_loss = np.mean(train_losses)
        val_loss = np.mean(val_losses)
        
        print(f'## Experiment results ({time_took:.2f}s) - {str(hyperparams)}')
        print(f'# CV Metrics - train: {train_loss:.5f}\t val: {val_loss:.5f}\t val_acc: (mean {np.mean(val_accuracies):.5f}) {str(val_accuracies)}')
        outputs.append((hyperparams, cv_metrics))

Begin Hyper-parameter Tuning -


### Running model (0.0001, 16, 'Adam', True)
Epoch 1/1, Train loss: 2.23815, Val loss: 2.10822
CV_1 (2.19s) -	 train_loss: 2.23815	 val_loss: 2.10822	 val_acc: 0.23400

Epoch 1/1, Train loss: 2.22117, Val loss: 2.09725
CV_2 (2.25s) -	 train_loss: 2.22117	 val_loss: 2.09725	 val_acc: 0.25470

Epoch 1/1, Train loss: 2.18520, Val loss: 2.32576
CV_3 (2.38s) -	 train_loss: 2.18520	 val_loss: 2.32576	 val_acc: 0.19520

Epoch 1/1, Train loss: 2.18423, Val loss: 2.26142
CV_4 (2.19s) -	 train_loss: 2.18423	 val_loss: 2.26142	 val_acc: 0.20870

Epoch 1/1, Train loss: 2.20537, Val loss: 2.23239
CV_5 (2.14s) -	 train_loss: 2.20537	 val_loss: 2.23239	 val_acc: 0.20060

## Experiment results (14.90s) - (0.0001, 16, 'Adam', True)
# CV Metrics - train: 2.20682	 val: 2.20501	 val_acc: (mean 0.21864) [0.234, 0.2547, 0.1952, 0.2087, 0.2006]


In [65]:
print(outputs)

[((0.0001, 16, 'Adam', True), ([2.2381500206835176, 2.221170012953086, 2.1851958137467906, 2.1842262844334037, 2.2053723065266966], [2.108224230188417, 2.097252578339957, 2.3257567537189514, 2.261423767114958, 2.2323944851865822], [0.234, 0.2547, 0.1952, 0.2087, 0.2006], [2.1857736110687256, 2.2517547607421875, 2.376272678375244, 2.189086675643921, 2.1445536613464355]))]
